<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خونه جدید‌؟
</font>
</h1>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در این تمرین می‌خواهیم به‌صورت عملی با نحوه‌ی پیاده‌سازی یک شبکه‌ی عصبی عمیق در <code>keras</code> آشنا شویم. کار خود را با خوانش داده‌ها و تحلیل اکتشافی آن شروع کرده، سپس با استفاده از پیش‌پردازش‌های مختلف، داده‌ها را به‌شکلی که مناسب استفاده در یک مدل یادگیری عمیق باشد آماده می‌کنیم و در نهایت به ساخت و آموزش یک شبکه‌ی عصبی می‌پردازیم. مجموعه‌داده‌ای که از آن استفاده خواهیم کرد شامل اطلاعاتی مربوط به حیوانات خانگی است. هدف این مسئله این است که پیش‌بینی کنیم آیا یک حیوان خانگی برای خود خانه‌ی جدیدی پیدا می‌کند یا خیر.
</font>
</p>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [ ]:
import keras
print(keras.__version__)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
مشخصات ویژگی‌های مجموعه‌داده‌ای که در دسترس شما قرار گرفته در جداول زیر آمده است.
ابتدا مجموعه‌داده‌ی آموزشی و آزمون را از دو فایل <code>csv</code> موجود در پوشه‌ی تمرین بخوانید.
<br>
    توجه داشته‌باشد در مجموعه‌داده‌ی آزمون، ستون <code>AdoptionSpeed</code> وجود ندارد، زیرا در ادامه از این ستون برای ساخت متغیر هدف مسئله استفاده خواهیم کرد.
</font>
</p>


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

|ستون|توضیحات|
|:------:|:---:|
|<code>Type</code>| نوع حیوان|
|<code>Age</code>| سن حیوان خانگی (به ماه) هنگام لیست شدن|
|<code>Breed1</code>|نژاد اصلی حیوان خانگی|
|<code>Gender</code>|جنسیت حیوان خانگی|
|<code>Color1</code>|رنگ اصلی حیوان خانگی|
|<code>Color2</code>|رنگ دوم حیوان خانگی (در صورت وجود)|
|<code>MaturitySize</code>|اندازه|
|<code>FurLength</code>|طول خز|
|<code>Vaccinated</code>|وضعیت واکسینه شدن|
|<code>Sterilized</code>|وضعیت انگل‌زدایی شدن|
|<code>Health</code>|وضعیت سلامتی|
|<code>Fee</code>|هزینه|
|<code>Description</code>|توضیحات|
|<code>PhotoAmt</code>|تعداد عکس بارگذاری شده|
|<code>AdoptionSpeed</code>|سرعت پذیرش (توضیح این اعداد در ادامه آمده است)|

<br>

|توضیحات|سرعت پذیرش|
|:------:|:---:|
|حیوان خانگی در همان روزی که لیست شده به سرپرستی پذیرفته شد.|<code>0</code>|
|	حیوان خانگی بین ۱ تا ۷ روز (هفته اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>1</code>|
|حیوان خانگی بین ۸ تا ۳۰ روز (ماه اول) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>2</code>|
|حیوان خانگی بین ۳۱ تا ۹۰ روز (ماه دوم و سوم) پس از لیست شدن به سرپرستی پذیرفته شد.|<code>3</code>|
|پس از ۱۰۰ روز لیست شدن، حیوان خانگی به سرپرستی پذیرفته نشد.|<code>4</code>|

</font>
</div>
</center>

In [ ]:
# Load the train and test data
train = None # TODO
test = None # TODO

In [ ]:
train.head()

In [ ]:
train.info()

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش داده‌ها
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همان‌طور که می‌دانید قبل از ساخت و آموزش مدل، برای آن‌که داده‌ها مناسب ورودی دادن به مدل باشند معمولاً نیاز است تغیراتی را بر روی مجموعه‌داده اعمال کنیم. گام نخست در پیش‌پردازش داده‌ها، بررسی مقادیر گم‌شده در مجموعه‌داده است که طبق خروجی سلول قبل مشاهده کردیم در این مجموعه‌داده، داده‌ی گم‌شده نداریم، بنابراین به سراغ گام‌های بعدی می‌رویم.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
<span style="color:red"><b>توجه:</b></span> جهت بررسی پیش‌پردازش انجام‌شده‌ بر روی مجموعه‌داده، دیتافرم <code>test</code> نیز مورد بررسی قرار خواهد گرفت، بنابراین لطفاً پیش‌پردازش یا تغییری بیشتر از آن‌چه از شما خواسته شده انجام ندهید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همان‌طور که گفتیم هدف این مسئله، پیش‌بینی پذیرش یا عدم پذیرش حیوان خانگی است، بنابراین باید در مجموعه‌داده‌‌ها یک ستون جدید با اسم <code>Target</code> اضافه کنیم تا از آن به‌عنوان متغیر هدف مسئله استفاده کنیم. مقادیر این ستون تنها در صورتی <code>True</code> است که حیوان خانگی پذیرش شده باشد. در این مرحله با استفاده ستون <code>AdoptionSpeed</code> ستون <code>Target</code> را ایجاد کنید.
<br>
به‌عنوان راهنمایی می‌توانید از دستور <code>np.where</code> استفاده کنید. البته توجه کنید که راه‌های متنوعی برای انجام این کار وجود دارد.
</font>
</p>




In [ ]:
train['Target'] = None # TODO

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
حال می‌توانیم ستون‌هایی که به آن‌ها نیاز نداریم را از دیتافریم حذف کنیم. با توجه به این‌که ستون <code>AdoptionSpeed</code> را به ستون <code>Target</code> تبدیل کردیم، این ستون را حذف می‌کنیم، همچنین در ستون <code>Description</code> اطلاعات متنی داریم که روش پردازش و کار با این نوع داده‌ها را در فصل‌های بعدی بررسی خواهیم کرد، بنابراین این ستون را نیز حذف می‌کنیم.
</font>
</p>



In [ ]:
train.drop(columns = [None, None], inplace = True) # TODO
test.drop(columns = [None], inplace = True) # TODO

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در گام بعد به‌سراغ ویژگی‌های دسته‌ای می‌رویم. ابتدا مجدد نگاهی به جزئیات دیتافریم بی‌اندازیم.
</font>
</p>



In [ ]:
train.info()

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در خروجی قطعه‌کد بالا می توان دید که ستون‌های
   <code>Fee</code>، <code>PhotoAmt</code> و <code>Age</code> مقادیر عددی دارند و برای باقی ستون‌های مجموعه‌داده نیاز است، پیش‌پردازش لازم برای تبدیل آن‌ها به مقادیر عددی را انجام دهیم. پس به بررسی تعداد دسته‌های هر ویژگی و نوع آن‌ها می‌پردازیم
</font>
</p>


In [ ]:
for column in ["Type", "Breed1", "Gender", "Color1", "Color2", "MaturitySize", "FurLength", "Vaccinated", "Sterilized", "Health"]:
    print("column:", column, "unique_values:", len(train[column].unique()))

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
 می‌دانیم ستون‌های <code>Health</code>، <code>FurLength</code> و <code>MaturitySize</code> از جنس متغیر‌های دسته‌ای ترتیبی هستند به این معنی که ترتیب خاصی بین مقادیر گروه‌ها وجود دارند. بنابراین می‌توانیم با استفاده از تابع <code>LabelEncoder</code> در کتابخانه‌ی <code>sklearn</code> به‌سادگی و بی‌دغدغه آن‌ها را به داده‌های عددی تبدیل کنیم.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
<span style="color:red"><b>توجه:</b></span> پیش‌پردازش‌هایی که بر روی نمونه‌های اعتبارسنجی/آزمون اعمال می‌کنید باید طبق آمارها یا اطلاعاتی باشد که از روی نمونه‌های آموزشی به دست آمده است. به‌عنوان مثال اگر می‌خواهید مقادیر گم‌شده‌ی یک ویژگی را با میانگین آن ویژگی پُر کنید، باید از میانگینی که طبق نمونه‌های آموزشی محاسبه‌شده استفاده کنید. به همین دلیل، در کدگذاری نیز برای نمونه‌های اعتبارسنجی/آزمون باید از تابع <code dir=ltr>transform()</code> به‌جای <code dir=ltr>fit_transform()</code> استفاده کنید.
</font>
</p>

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train['MaturitySize'] = None # TODO
test['MaturitySize'] = None # TODO

train['FurLength'] = None # TODO
test['FurLength'] = None # TODO

train['Health'] = None # TODO
test['Health'] = None # TODO

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
اما ستون‌های <code>Type</code>، <code>Breed1</code>، <code>Gender</code>، <code>Color1</code> ،<code>Color2</code> <code>Vaccinated</code> و <code>Sterilized</code>  از جنس متغیر‌های دسته‌ای اسمی هستند، به این معنی که ترتیب خاصی بین مقادیر گروه‌ها وجود ندارد. بنابراین برای این ویژگی‌ها نمی‌توانیم از <code>LabelEncoder</code> استفاده کنیم، زیرا این نوع داده‌ها از نظر ترتیبی تفاوتی با یک‌دیگر ندارند و اگر به‌سادگی به هر کدام از این ویژگی‌ها یک عدد اختصاص دهیم آن‌گاه به‌نوعی بین آن‌ها ترتیب قائل شدیم که نادرست است. برای حل این مسئله از روش کدگذاری دودویی استفاده می‌کنیم، به این معنی که به‌طور مثال به ازای ستون <code>color1</code> که ۷ مقدار یکتا <code dir=ltr>['Black', 'Brown', 'Cream', 'Gray', 'Golden', 'White', 'Yellow']</code> را دارد ۳ ستون اضافه می‌کنیم زیرا در هر ستون می‌توانیم مقدار ۱ و ۰ داشته باشیم و برای این‌که ۷ دسته را پوشش دهیم حداقل به ۳ ستون نیاز داریم (<code dir=ltr>7 < 3^2</code>). حال به هرکدام از این دسته‌ها عددی اختصاص می‌دهیم و ستون‌های مربوط به کد دودویی آن عدد را پر می‌کنیم.
<br>
    تابع <code>BinaryEncoder</code> از کتابخانه‌ی <code>category_encoders</code> این عملیات را انجام می‌دهد.
        <br>
    توجه داشته باشید در ستون <code>Gender</code> و <code>Type</code> به علت این‌که فقط ۲ دسته داریم می‌توانیم با تابع <code>LabelEncoder</code> تبدیل را انجام دهیم.
    </font>
</p>

In [ ]:
train['Type'] = None # TODO
test['Type'] = None # TODO

train['Gender'] = None # TODO
test['Gender'] = None # TODO

In [ ]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder()

train_binary_Breed1 = None # TODO
test_binary_Breed1 = None # TODO

train_binary_Color1 = None # TODO
test_binary_Color1 = None # TODO

train_binary_Color2 = None # TODO
test_binary_Color2 = None # TODO

train_binary_Vaccinated = None # TODO
test_binary_Vaccinated = None # TODO

train_binary_Sterilized = None # TODO
test_binary_Sterilized = None # TODO

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
حال ستون‌های ساخته شده برای هر متغیر را به مجموعه‌داده اصلی اضافه می‌کنیم و این ستون‌ها را حذف می‌کنیم.
</font>
</p>

In [ ]:
train = pd.concat([train, train_binary_Breed1, train_binary_Color1, train_binary_Color2, train_binary_Vaccinated, train_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
train.drop(columns = columns, inplace = True)

test = pd.concat([test, test_binary_Breed1, test_binary_Color1, test_binary_Color2, test_binary_Vaccinated, test_binary_Sterilized], axis=1)
columns = ['Breed1','Color1','Color2','Vaccinated','Sterilized']
test.drop(columns = columns, inplace = True)

train.info()

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
نرمال سازی داده‌ها
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
 یکی از اعمال مهم پیش‌پردازش، تغییر مقیاس ویژگی‌ها به‌صورت توزیع نرمال است که اصطلاحاً به آن نرمال‌سازی (Normalization) می‌گوییم.
 نرمال‌سازی به کاهش تغییرات شدید در وزن‌ها کمک کرده و باعث سرعت بالاتر همگرایی مدل می‌شود.
برای انجام نرمال‌سازی در این تمرین باید اعداد هر ویژگی را طوری تغییر دهیم که میانگین آن‌ها <code>0</code> شود و واریانس <code>1</code> داشته باشند، این‌کار را می‌توانیم با استفاده از فرمول زیر انجام دهیم.
برای سری داده‌ی <code>X</code> (که در اینجا مقادیر هر ستون می‌باشد) به طوری‌که <code>X = [x_1, x_2, ..., x_n]</code>، با کم کردن میانگین از هر نمونه‌ی داده (<code>x_i</code>) و تقسیم آن بر انحراف معیار داده‌ها (سیگما) می‌توانیم سری داده‌ی نرمال‌شده را به‌دست آوریم.
</font>
</p>

$$ Z = \frac{x_i - \bar{x}}{\sigma} $$

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
دقت داشته باشید، از آنجا که در واقع ما فقط اطلاعات مربوط به مجموعه‌داده‌ی آموزش را در هنگام ساخت مدل داریم، برای نرمال‌سازی نمونه‌های آزمون نیز باید از میانگین و انحراف معیار نمونه‌های آموزش استفاده کنیم. همچنین حتما از تابع میانگین و انحراف معیار کتابخانه <code>pandas</code> استفاده کنید.
</font>
</p>

In [ ]:
for column in train.columns:
  mean = None # TODO (Mean of train_data[column])
  std = None # TODO (Standard Deviation of train_data[column])
  train[column] = (train[column]- mean)/std
  test[column]=  (test[column] - mean)/std

test.head()

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
همان‌طور که در درسنامه‌های قبلی گفتیم برای ارزیابی و بهبود مدل بخشی از داده‌های آموزش را به‌عنوان داده‌های اعتبارسنجی جدا می‌کنیم. برای این‌کار می‌توانید از تابع <code>train_test_split</code> کتابخانه‌ی <code>sklearn</code> استفاده کنید. مستندات این تابع را می‌توانید در  <a href=https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html> این لینک </a> پیدا کنید. توجه داشته باشید در این مسئله <code>0.1</code> از داده‌های آموزش را به داده‌های اعتبارسنجی اختصاص می‌دهیم و <code>0.9</code> باقی مانده را برای آموزش مدل استفاده می‌کنیم.

</font>
</p>




In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = None # TODO

print('Train examples:', len(X_train), len(y_train))
print('Validation examples:', len(X_valid), len(y_valid))
print('Test examples:',  len(test))

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ساخت شبکه‌ی عصبی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
پس از انجام پیش‌پرداز‌های مورد نیاز نوبت به ساخت و آموزش مدل رسیده است.
در این مسئله از شما می‌خواهیم ابتدا یک لایه ورودی با ابعاد مناسب مسئله که همان تعداد ویژگی‌های مجموعه‌داده است بسازید. سپس ۳ لایه‌ی متراکم با تعداد نورون‌های ۵۰۰۰، ۱۰۰۰ و ۵۰۰ (به ترتیب از راست به چپ) با تابع‌ فعال‌ساز  <code>relu</code> ایجاد کنید و در نهایت یک لایه‌ی متراکم خروجی با تعداد نورون مناسب و تابع فعال‌ساز  <code>sigmoid</code> اضافه کنید.
</font>
</p>

In [ ]:
model = None # TODO : define a Sequential model
model.add(keras.layers.Input(None)) # TODO : add Input Layer with proper Shape
model.add(keras.layers.Dense(None)) # TODO : add Dense Layer with 5000 neurons and relu activation function
model.add(keras.layers.Dense(None)) # TODO : add Dense Layer with 1000 neurons and relu activation function
model.add(keras.layers.Dense(None)) # TODO : add Dense Layer with 500 neurons and relu activation function
model.add(keras.layers.Dense(None)) # TODO : add Dense Layer as Output Layer with sigmoid activation function

In [ ]:
model.summary()

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش شبکه‌ی عصبی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در ادامه شبکه را با تابع هزینه و بهینه‌ساز تعریف‌شده <code>compile</code> می‌کنیم. (درباره‌ی توابع هزینه‌ و بهینه‌سازهای مختلف در فصل بعد کامل صحبت خواهیم کرد)</font>
</p>

In [ ]:
model.compile(optimizer = 'adam',
              loss = keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = ['accuracy'])

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <ul dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir">
در این مرحله باید مدل ساخته شده را با اندازه‌ی دسته ۱۲۸ (<code>BATCH_SIZE</code>) و به تعداد دور ۱۰ (<code>epochs</code>) آموزش دهید.
</font>
</p>




In [ ]:
epochs = None # TODO
BATCH_SIZE = None # TODO

history = model.fit(None, None,
                    batch_size=BATCH_SIZE,
                    epochs=epochs,
                    validation_data=(None, None)) # TODO



<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌بینی برای داده‌های آزمون و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
شبکه‌ی خود را روی مجموعه‌داده‌ی <code>test</code>  آزمایش کنید و خروجی را در یک دیتافریم (<code>dataframe</code>) ذخیره کنید. دقت کنید مقادیر مجاز مقادیر دودویی (<code>False</code>/<code>True</code>) می‌باشد.
</font>
</p>

<h3 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
توجه داشته باشید معیار ارزیابی مدل شما در این تمرین معیار <code>F1 score</code> است، که می‌توانید مستندات آن را در <a href = 'https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html'>این لینک</a> مطالعه نمایید. لازم به ذکر است که از حالت میانگین‌گیری <code>"weighted"</code> این معیار استفاده خواهد شد.
</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>70</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
  
|ستون|توضیحات|
|:------:|:---:|
|<code>Target</code>|آیا حیوان‌خانگی خانه جدید پیدا کرده است یا خیر|
  
</font>
</div>
</center>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    اسم دیتافریم باید <code>submission</code> باشد؛ در غیر این صورت، سامانه داوری نمی‌تواند تلاش‌ شما را ارزیابی کند.
    <br>
    این دیتافریم تنها شامل ۱ ستون با اسم <code>Target</code> است و ۱۰۰۰ سطر دارد.
    <br>
    به ازای هر سطر موجود در دیتافریم <code>test</code> شما باید یک مقدار پیش‌بینی‌شده داشته باشید.
    <br>
    جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته در جواب شما، اعداد ستون <code>Target</code> ممکن است متفاوت باشد و این فقط یک مثال است.
</font>
</p>

<center>
<div dir=ltr style="direction: ltr;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
||<code>Target</code>|
|:----:|:-----:|
|0|<code>True</code>|
|1|<code>True</code>|
|2|<code>False</code>|
|3|<code>False</code>|
|4|<code>True</code>|

</font>
</div>
</center>

In [ ]:
submission = # TODO

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد. همچنین اگر از گوگل کولب استفاده می‌کنید، در صورت نیاز به پشتیبانی حتماً آخرین نسخه از نت‌بوک را به‌صورت دستی دانلود کرده و داخل فایل ارسالی قرار دهید یا لینک کولب را با ما به‌اشتراک بگذارید.
</font>
</p>

In [ ]:
import zipfile
import joblib
import numpy as np
import json
import os

if not os.path.exists(os.path.join(os.getcwd(), 'new_home.ipynb')):
    %notebook -e new_home.ipynb

test.to_csv("test.csv")

model_info=[]
for layer in model.layers:
  if layer.__class__.__name__=="Dense":
    model_info.append({
        "name": layer.__class__.__name__,
        "units":layer.units,
        "activation":layer.get_config()["activation"]
    })
  else:
    model_info.append({"name": layer.__class__.__name__})

with open("model_info.json", "w") as f:
    json.dump(model_info, f)


def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)

file_names = ["test.csv",'submission.csv', 'model_info.json', 'new_home.ipynb']
compress(file_names)